# Combine *Cluster Failure* False Positive Maps
1. Flirt AFNI and SPM false cluster maps into MNI space (FSL maps are already there).
2. Mask maps with FSL's MNI152 2mm brain mask.
3. Weight values by prevalence of the package in the literature.
    - I just searched *fmri afni*, *fmri fsl*, and *fmri spm* in Google Scholar (on 170311) to get the estimated prevalence of each package.
    - Instead of looking into the individual tools within each package, I just divided the package number by the number of tools within the package (1 or 2).
4. Combine weighted maps (just add them together).
5. Normalize combined map so that each voxel's value is a probability.

In [1]:
from __future__ import division
import numpy as np
import nibabel as nib
from glob import glob
from os.path import join, basename, splitext
from nipype.interfaces import fsl
import shutil
import gzip

In [2]:
par_dir = '/home/tsalo006/Templates/'
orig_dir = join(par_dir, 'original_images/')
flirt_dir = join(par_dir, 'flirted_images/')
mask_dir = join(par_dir, 'masked_images/')
gz_dir = join(par_dir, 'gz_images/')

In [3]:
mask = nib.load('/home/applications/fsl/5.0.8/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')
mask = mask.get_data().astype(float)

In [4]:
fs = sorted(glob(join(orig_dir, '*.nii')))
all_data = [[] for _ in range(len(fs))]

# Relative frequency of each tool in the literature.
tools = ['AFNIMEMA', 'AFNIOLS', 'FSLFLAME', 'FSLOLS', 'SPM']
freqs = [11000./2,   11000./2,  15700./2,   15700./2, 46600.]
freqs = freqs / np.sum(freqs)

for i, f in enumerate(fs):
    fname = basename(splitext(f)[0])
    gz_f = join(gz_dir, fname+'.nii.gz')
    
    flirted_f = join(flirt_dir, '{0}.nii.gz'.format(fname))
    masked_f = join(mask_dir, '{0}.nii.gz'.format(fname))
    
    # Gzip original files.
    with open(f, 'rb') as fi, gzip.open(gz_f, 'wb') as fo:
        shutil.copyfileobj(fi, fo)
    
    if 'FSL' not in fname:
        # FLIRT AFNI and SPM maps to MNI.
        flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
        flt.inputs.in_file = gz_f
        flt.inputs.reference = '/home/data/nbc/tools/spm8/templates/EPI.nii'
        flt.inputs.out_file = flirted_f
        flt.inputs.output_type = 'NIFTI_GZ'
        flt.run()
    else:
        # FSL maps already in MNI space.
        shutil.copyfile(gz_f, flirted_f)

    # Mask each map with the MNI152 brain mask.
    dat = nib.load(flirted_f)
    data = dat.get_data()
    data *= mask
    img = nib.Nifti1Image(data, dat.affine)
    img.to_filename(masked_f)
    
    # Weight each map by tool's relative popularity.
    # Combine maps.
    for j, tool in enumerate(tools):
        if tool in fname:
            if j == 0:
                combined = data * freqs[j]
            else:
                combined += data * freqs[j]

# Normalize.
combined = combined / np.sum(combined)

# Write out combined map.
img = nib.Nifti1Image(combined, dat.affine)
img.to_filename('falseclusters_combined.nii.gz')